<h1>STA 220 Assignment 1</h1>

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import json
import os
import ndjson
import pandas
from datetime import datetime
import concurrent.futures
from dotenv import load_dotenv

<h2>Exercise 1</h2>

In [2]:
db = sql.connect("./data/lahmansbaseballdb.sqlite")

In [3]:
db

<h3>(a, i) Which pitcher has the second most home runs allowed in the American League?</h3>

In [4]:
query_second_most_hra = """
SELECT p.nameFirst, p.nameLast, SUM(pi.HR) as hra
FROM Pitching pi
JOIN People p ON pi.playerID = p.playerID
WHERE pi.lgID = 'AL'
GROUP BY pi.playerID
ORDER BY hra DESC
LIMIT 1 OFFSET 1;
"""

In [5]:
second_most_hra = pd.read_sql(query_second_most_hra, db).squeeze()

In [6]:
print(second_most_hra['nameFirst'] + " " + second_most_hra['nameLast'] + " (" + str(second_most_hra["hra"]) + ")")

Tim Wakefield (401)


<b>The pitcher with the second most home runs allowed in American League is Tim Wakefield with the home runs allowed as 401</b>

<h3> (a,ii) Which pitcher has the worst home runs allowed per game ratio?</h3>

In [7]:
query_worst_hra_to_game = """
SELECT pi.playerID, SUM(pi.HR) as hra, SUM(pi.G) as games, SUM(pi.HR) / CAST(SUM(pi.G) AS REAL) 
as hra_per_game, pe.nameFirst,pe.nameLast FROM 
pitching AS pi LEFT JOIN people AS pe ON pi.playerID=pe.playerID 
WHERE lgID='AL' GROUP BY pi.playerID HAVING games>0 ORDER BY hra_per_game DESC;
"""

In [8]:
worst_hra_to_game = pd.read_sql(query_worst_hra_to_game, db).squeeze().iloc[0]

In [9]:
print(worst_hra_to_game['nameFirst'] + " " + worst_hra_to_game['nameLast'] + " (" + str(worst_hra_to_game["hra_per_game"]) + ")")

Ryan Snare (3.0)


<b>There were multiple people with the worst home runs allowed per game ratio, choosing the one with index 0</b>

<b>One of the pitcher with the worst home runs allowed per game ratio is Ryan Snare with a ratio of 3.0

<h3>(b, i) Amongst all players in the American League that have passed, report their average lifespan in full years.</h3>

For the average lifespan: The query subtracts the birthYear from the deathYear and adjusts for the birth and death month/day. If a person died before their birthday in their death year, it subtracts one year from their age. This is used to calculate the average lifespan. All players have an entry in the Batting table has the record of all professional players.

In [10]:
query_avg_lifespan = """
SELECT AVG((deathYear - birthYear) + 
           CASE 
               WHEN deathMonth < birthMonth THEN -1 
               WHEN deathMonth > birthMonth THEN 0 
               WHEN deathDay < birthDay THEN -1 
               ELSE 0 
           END) as avg_lifespan
FROM People p
JOIN Batting b ON p.playerID = b.playerID
WHERE p.deathYear IS NOT NULL AND b.lgID = 'AL';
"""

In [11]:
avg_lifespan = pd.read_sql(query_avg_lifespan, db).squeeze()
avg_lifespan

70.69888436525142

Rounding off the average lifespan

In [12]:
round(avg_lifespan)

71

<b>The average lifespan in full years is 71 years!</b>

<h3>(b,ii) Return the six schools with most hall-of-fame alumni.</h3>

Referred Readme.txt and Microsoft Copilot to form the correct query.

In [13]:
query_top_six_schools_hall_of_fame = """
SELECT s.name_full, COUNT(DISTINCT h.playerID) as num_hof
FROM schools s
JOIN CollegePlaying cp ON s.schoolID = cp.schoolID
JOIN HallOfFame h ON cp.playerID = h.playerID
WHERE h.inducted = 'Y'
GROUP BY s.schoolID
ORDER BY num_hof DESC
LIMIT 6;
"""

In [14]:
top_six_schools_hall_of_fame = pd.read_sql(query_top_six_schools_hall_of_fame, db)

In [15]:
print("The six schools with most hall-of-fame alumni are:")
top_six_schools_hall_of_fame

The six schools with most hall-of-fame alumni are:


,name_full,num_hof
0,University of Southern California,3
1,St. Bonaventure University,2
2,San Diego State University,2
3,University of Minnesota,2
4,University of Michigan,2
5,"Miami-Dade College, North Campus",2


<h3> <b>(b,iii)</b> What fraction of managers have not been professional players?</h3>

Referred Readme.txt and Microsoft Copilot to form the correct query.

All professional players have an entry in Batting table.

In [16]:
query_managers_not_pro_players = """
SELECT COUNT(DISTINCT m.playerID) * 1.0 / (SELECT COUNT(DISTINCT playerID) FROM Managers) as fraction_non_players
FROM Managers m
LEFT JOIN Batting b ON m.playerID = b.playerID
WHERE b.playerID IS NULL;
"""

In [17]:
manager_count = pd.read_sql(query_managers_not_pro_players, db).squeeze()
print(f"Fraction of managers who have not been professional players: {manager_count}")

Fraction of managers who have not been professional players: 0.17507002801120447


<b>The fraction of managers who have not been professional players is 0.17507002801120447</b>

<h3><b>(c)</b> Create a world map with a color gradient corresponding to the log-number of players per country in the data set. Use pandas.read_html to retrieve the ISO codes from wikipedia, and merge those records as good as you can. Match no more than ten countries to their ISOs manually.</h3>

In [18]:
table_birthCountry = table = pd.read_sql("SELECT birthCountry,COUNT(*) as count from people GROUP BY birthCountry", db)

In [22]:
iso_data = pd.read_html("https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes", header=1)[0]
iso_country = iso_data['ISO 3166[1] name[5]']
iso_codes = iso_data['A-3 [5]']

     ISO 3166[1] name[5]        World Factbook[6] official state name[a]  \
0            Afghanistan             The Islamic Republic of Afghanistan   
1          Åland Islands                                           Åland   
2                Albania                         The Republic of Albania   
3                Algeria     The People's Democratic Republic of Algeria   
4         American Samoa                 The Territory of American Samoa   
..                   ...                                             ...   
266    Wallis and Futuna  The Territory of the Wallis and Futuna Islands   
267  Western Sahara [aj]            The Sahrawi Arab Democratic Republic   
268                Yemen                           The Republic of Yemen   
269               Zambia                          The Republic of Zambia   
270             Zimbabwe                        The Republic of Zimbabwe   

    Sovereignty [6][7][8] A-2 [5] A-3 [5] Num. [5]  \
0               UN member      AF

In [ ]:
iso_country

In [ ]:
iso_codes

Converting iso_country and iso_codes into Dataframe and cleaining it

In [ ]:
df_country = pd.DataFrame({'ISO Country': iso_country, 'ISO Code': iso_codes})

#Removing square brackets and the content inside it
df_country['ISO Country'] = df_country['ISO Country'].str.replace(r'\[.*\]', '', regex=True)  
df_country['ISO Country'] = df_country['ISO Country'].str.replace(r'\(.*\)', '', regex=True)
df_country['ISO Country'] = df_country['ISO Country'].str.strip() 

Merging the wikipedia data and the data from the db. Referred to copilot for below code and map creation

In [ ]:
merged_country = pd.merge(table_birthCountry, df_country, how='left', left_on='birthCountry', right_on='ISO Country')

In [ ]:
merged_country['log_player_count'] = np.log(merged_country['count'])
merged_country.head(57)

Handling nan rows

In [ ]:
nan_rows = merged_country[pd.isna(merged_country).any(axis=1)]

<b>Mapping unmapped countries manually</b>

In [ ]:
merged_country.loc[merged_country['birthCountry'] == 'Curacao', 'ISO Code'] = 'CUW'
merged_country.loc[merged_country['birthCountry'] == 'D.R.', 'ISO Code'] = 'DOM'
merged_country.loc[merged_country['birthCountry'] == 'Netherlands', 'ISO Code'] = 'NLD'
merged_country.loc[merged_country['birthCountry'] == 'Russia', 'ISO Code'] = 'RUS'
merged_country.loc[merged_country['birthCountry'] == 'USA', 'ISO Code'] = 'USA'
merged_country.loc[merged_country['birthCountry'] == 'United Kingdom', 'ISO Code'] = 'GBR'
merged_country.loc[merged_country['birthCountry'] == 'V.I.', 'ISO Code'] = 'VIR'
merged_country.loc[merged_country['birthCountry'] == 'P.R.', 'ISO Code'] = 'PRI'
merged_country.loc[merged_country['birthCountry'] == 'CAN', 'ISO Code'] = 'CAN'
merged_country.loc[merged_country['birthCountry'] == 'Czech Republic', 'ISO Code'] = 'CZE'

In [ ]:
world_map = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
merged_world_map = world_map.merge(merged_country,how='left', left_on='iso_a3', right_on='ISO Code')

<b>WORLD MAP WITH COLOR GRADIENT</b>

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 7))
merged_world_map.boundary.plot(ax=ax)
merged_world_map.plot(column='log_player_count', cmap='gist_heat', linewidth=0.10, ax=ax, edgecolor='0.100', legend=True)

plt.title('Country vs Log Number of Players')
plt.show()

Closing db connection

In [ ]:
db.close()

<h2>Exercise 2</h2>

<h3> <b>(a)</b> What is the real name of the leader of the blitz leaderboard?</h3>

Querying the blitz leaderboard and the top player's username.

In [ ]:
top_blitz_player = requests.get("https://lichess.org/api/player/top/1/blitz")
top_blitz_player.raise_for_status

In [ ]:
json_blitz_leader = json.loads(top_blitz_player.text)
username = json_blitz_leader['users'][0]['username']
print(f'The username of the top blitz player is {username}')

Using the username to fetch the user data from lichess

In [ ]:
user_data = requests.get("https://lichess.org/api/user/" + username)
user_data.raise_for_status
user_data.text

In [ ]:
json_user_data = json.loads(user_data.text)
actual_name = json_user_data['profile']['firstName'] + " " + json_user_data['profile']['lastName']
print(f'The real name of the leader of the blitz leaderboard is {actual_name}')

<b>The real name of the leader of the blitz leaderboard is Vladislav Artemiev</b>

<h3> <b>(b, i)</b> Get the username of the last player that played a rapid game against user athena-pallada. </h3>

I have added the Authorization Bearer token (my personal access token) as an environment variable and fetching the environment and the api token

In [ ]:
load_dotenv()

In [ ]:
api_token = os.environ.get('API_TOKEN')

In [ ]:
lichess_export_games_user_url = "https://lichess.org/api/games/user/athena-pallada"
query_params = {
    'perfType' : "rapid",
     'max' : 1,
    'moves' : False,
    'sort' : "dateDesc"
}
response_user_game = requests.get(lichess_export_games_user_url, params = query_params, headers={"Accept":"application/x-ndjson", "Authorization": f'Bearer {api_token}'})
response_user_game.raise_for_status

Since the response is in ndjson format (streaming the data), using below method to convert it into a json object

In [ ]:
def parse_ndjson(ndjson):
    resp_json = []
    for json_obj in ndjson:
        if json_obj:
            resp_json.append(json.loads(json_obj))
    return resp_json

In [ ]:
ndjson_user_game = response_user_game.content.decode().split('\n')
json_user_game = parse_ndjson(ndjson_user_game)

Converting the json data into Dataframes by using normalization for handling multiple indexed columns and easier access to data

In [ ]:
df_user_game = pd.json_normalize(json_user_game)
df_user_game.info()

athena-pallada can be either black or white, based on that checking the opponent of athena-pallada

In [ ]:
for index, row in df_user_game.iterrows():
    if row['players.white.user.name'] == 'athena-pallada':
       print(f"The opponent of athena-pallada is: {row['players.black.user.name']}")
    else:
        print(f"The opponent of athena-pallada is: {row['players.white.user.name']}")

<h3><b>(b,ii)</b> In all games against this user, how many times did athena-pallada win? (Provide code that answers the question in case more than just a single game is returned)</h3>

In [ ]:
user_game_url_vs_Bacio129 = "https://lichess.org/api/games/user/athena-pallada"
query_params = {
     'moves' : False,
     'vs' : "Bacio129"
}
response_game_data_vs_Bacio129 = requests.get(user_game_url_vs_Bacio129, params = query_params, headers={"Accept":"application/x-ndjson", "Authorization": f'Bearer {api_token}'})

In [ ]:
ndjson_game_data_vs_Bacio129 = response_game_data_vs_Bacio129.content.decode().split('\n')
json_game_data_vs_Bacio129 = parse_ndjson(ndjson_game_data_vs_Bacio129)

In [ ]:
df_game_data_vs_Bacio129 = pd.json_normalize(json_game_data_vs_Bacio129)
df_game_data_vs_Bacio129.info()

Since we got two games, the below method extracts the games in which athena-pallada won. It checks if athena-pallada is white or black and correspondingly the winner of that game. Referred copilot for below code.

In [ ]:
wins_athena = df_game_data_vs_Bacio129[(df_game_data_vs_Bacio129['players.white.user.name'] == 'athena-pallada') 
& (df_game_data_vs_Bacio129['winner'] == 'white')].shape[0]

wins_athena += df_game_data_vs_Bacio129[(df_game_data_vs_Bacio129['players.black.user.name'] == 'athena-pallada') & 
(df_game_data_vs_Bacio129['winner'] == 'black')].shape[0]

print(f"athena-pallada has won {wins_athena} games.")

<h3>Consider the top ten players in the bullet leaderboard.<br> 
<b>(i) Which player has the most bullet games overall?<br>
(ii) Which player has played the most bullet games relative to account age in days?<br> 
(iii) Which player has the worst win-to-loss ratio over all formats?</b></h3>

The below method fetches the performance statistics and public user data for each username in the top 10 players in bullet leaderboard. It also calcultes the required statistics and puts it a list.

In [ ]:
def fetch_stats(username):
    response = requests.get("https://lichess.org/api/user/" + username + "/perf/bullet")
    player_stats = json.loads(response.text)

    response = requests.get("https://lichess.org/api/user/" + username)
    public_data = json.loads(response.text)
    
    # Getting bullet games count
    bullet_games = player_stats['stat']['count']['all']

    #Getting account age
    created_at_timestamp = public_data['createdAt']

    # Converting the timestamp to datetime
    created_at = datetime.fromtimestamp(created_at_timestamp / 1000)

    # Calculating account age in days
    account_age = (datetime.now() - created_at).days
    
    #Calculating bullet games win to account age
    bullet_games_to_account_age = bullet_games/account_age
    
    win_to_loss_ratio = public_data['count']['win']/public_data['count']['loss']
    
    return [username, bullet_games, bullet_games_to_account_age, win_to_loss_ratio]

Calling each user stats from the top 10 players concurrently

In [ ]:
response_top_10= requests.get("https://lichess.org/api/player/top/10/bullet")
top_10_players = json.loads(response_top_10.text)

stats = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    stats = list(executor.map(fetch_stats, [player['username'] for player in top_10_players['users']]))

In [ ]:
# Create a DataFrame from the data
df = pd.DataFrame(stats, columns=['username', 'bullet_games', 'bullet_games_to_account_age','win_to_loss_ratio'])

# Sorting each stat
df = df.sort_values(by='bullet_games', ascending=False)
player_most_bullet_games = df.iloc[0]['username']

df = df.sort_values(by='bullet_games_to_account_age', ascending=False)
player_most_bullet_games_to_age = df.iloc[0]['username']

df = df.sort_values(by='win_to_loss_ratio', ascending = True)
player_worst_wint_to_loss = df.iloc[0]['username']

print(f"The player with the most bullet games overall is {player_most_bullet_games}.")
print(f"The player with the most bullet games relative to account age is {player_most_bullet_games_to_age}.")
print(f"The player with the worst win to loss ratio is {player_worst_wint_to_loss}.")

<h3><b>(d)</b> Get all games from user manwithavan. Group them by opening and print the ten most popular.</h3>

In [ ]:
url_user_game_data = "https://lichess.org/api/games/user/manwithavan"
query_params = {
     'moves' : False,
     'opening' : True
}
response_activity = requests.get(url_user_game_data , params = query_params, headers={"Accept":"application/x-ndjson", "Authorization": f'Bearer {api_token}'})

In [ ]:
ndjson_activity = response_activity.content.decode().split('\n')
json_activity = parse_ndjson(ndjson_activity)

In [ ]:
df_activity = pd.json_normalize(json_activity)
df_activity.info()

Group the openings by name and printing the value counts

In [ ]:
df_activity['opening.name'].value_counts().head(10)